In [1]:
%matplotlib inline

# 신경망 구축하기

신경망은 데이터에 대한 작업을 수행하는 레이어/모듈로 구성됩니다. [torch.nn](https://pytorch.org/docs/stable/nn.html)은 신경망을 구축하는 데 필요한 모든 구성 요소를 제공하며, PyTorch의 모든 모듈은 [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)을 서브클래싱(subclassing)합니다. 신경망은 다른 모듈(레이어)로 구성된 모듈 자체이며, 중첩 구조를 사용하면 복잡한 아키텍처(architectures)를 쉽게 구축하고 관리할 수 있습니다.

다음 섹션에서는 FashionMNIST 데이터셋으로 이미지를 분류하는 신경망을 구축합니다.

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 하드웨어 장치 얻기

GPU와 같은 하드웨어 가속기에서 모델 훈련을 원할 경우, [torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html)를 통해 GPU 사용 여부를 확인할 수 있습니다.

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


## 클래스 정의하기

`nn.Module`을 서브클래싱하여 신경망을 정의하고, `__init__`에서 신경망 계층(Layer)를 초기화합니다. 모든 `nn.Module`의 서브클래스는 `forward` 메소드의 입력 데이터에 대한 작업을 구현합니다.

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

`NeuralNetwork`의 인스턴스(instance)를 만들고, `device`로 옮긴 후 구조를 출력합니다.


In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


모델을 사용하기 위해 입력 데이터를 전달하며, 이 작업은 일부 [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866)과 함께 모델의 `foward`를 실행합니다.
이때 `model.forward()`를 직접 호출하지 않도록 주의하십시오.

입력에서 모델을 호출하면 각 클래스에 대한 원시 예측값이 포함된 10차원 Tensor를 반환하며, `nn.Softmax` 모듈의 인스턴스를 통해 예측 확률을 얻습니다.

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) 
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


## 모델 계층(Model layers)

Fashion MNIST 모델의 레이어를 분해해보겠습니다. 이를 설명하기 위해 **28x28** 크기의 이미지 3개를 미니배치로 가져와 네트워크를 통과할 때 어떤 일이 발생하는지 확인합니다.

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

각 28x28의 2D 이미지를 784 픽셀 값의 연속 배열(Contiguous Array)로 변환하기 위해 [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)를 초기화합니다.(미니배치의 차원(dim=0)은 유지됨)

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear 

[linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)는 저장된 가중치(weights)와 편향(biases)을 사용하여 입력에 선형 변환을 적용하는 모듈입니다.

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

비선형 활성화 함수는 모델의 Input과 Output 사이에 복잡한 매핑(mapping)을 생성하며, 선형 변환 후에 적용되어 비선형성을 통해 신경망이 다양한 패턴을 학습하도록 돕습니다.


이 모델에서는 선형 Layer 간에 [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)를 사용하지만, 모델에 비선형성을 도입하기 위한 다른 활성화 함수도 존재합니다.

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2190,  0.1448, -0.5783,  0.1782, -0.4481, -0.2782, -0.5680,  0.1347,
          0.1092, -0.7941, -0.2273, -0.4437,  0.0661,  0.2095,  0.1291, -0.4690,
          0.0358,  0.3173, -0.0259, -0.4028],
        [-0.3531,  0.2385, -0.3172, -0.4717, -0.0382, -0.2066, -0.3859,  0.2607,
          0.3626, -0.4838, -0.2132, -0.7623, -0.2285,  0.2409, -0.2195, -0.4452,
         -0.0609,  0.4035, -0.4889, -0.4500],
        [-0.3651, -0.1240, -0.3222, -0.1072, -0.0112, -0.0397, -0.4105, -0.0233,
         -0.0342, -0.5680, -0.4816, -0.8085, -0.3945, -0.0472,  0.0247, -0.3605,
         -0.0347,  0.1192, -0.2763,  0.1447]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.2190, 0.1448, 0.0000, 0.1782, 0.0000, 0.0000, 0.0000, 0.1347, 0.1092,
         0.0000, 0.0000, 0.0000, 0.0661, 0.2095, 0.1291, 0.0000, 0.0358, 0.3173,
         0.0000, 0.0000],
        [0.0000, 0.2385, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2607, 0.3626,
         0.0000, 0.0000, 0.0000, 0.0000, 0.2409, 0.000

### nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)은 모듈의 컨테이너로, 데이터는 정의된 순서대로 모든 모듈을 통과합니다. 순차형(Sequential) 컨테이너를 통해 `seq_modules`와 같은 빠른 네트워크를 구성할 수 있습니다.

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

신경망의 마지막 선형 Layer는 `logits`을 반환하며, 범위는 \[`-infty`, `infty`\]입니다. 이후 [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)을 거쳐 `logits`은 각 클래스에 대한 모델의 예측 확률을 나타내는 값인 \[0, 1\]로 스케일(Scale)되며, 이때 `dim` 매개 변수는 값의 합이 1이 되는 차원을 나타냅니다.

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## 모델 파라미터(Model parameters)

신경망 내부의 많은 레이어는 *parameterzied* 되어 있으며, 훈련 중 최적화된 가중치와 편향을 가지고 있습니다. `nn.Module`을 서브클래싱하면 모델 객체 내에 정의된 모든 필드(Fileds)를 자동으로 추적하고, 모델의 `parameters()` 또는 `named_parameters()` 메소드를 사용하여 모든 매개 변수(parameter)에 접근할 수 있습니다.

아래 예제에서는 각 매개 변수의 개수와 값에 대한 미리보기를 출력합니다.

In [13]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0320,  0.0326, -0.0032,  ..., -0.0236, -0.0025, -0.0175],
        [ 0.0180,  0.0271, -0.0314,  ..., -0.0094, -0.0170, -0.0257]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0134,  0.0036], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0262,  0.0072, -0.0348,  ..., -0.0374,  0.0345,  0.0374],
        [ 0.0439, -0.0101,  0.0218,  ..., -0.0419,  0.0212, -0.0081]],
       device='cuda:0', grad_fn=<SliceBack

--------------


